In [1]:
import urllib
import pandas as pd
import os
import numpy as np
import datetime as dt
from tqdm.contrib.concurrent import process_map
import requests
import pickle

l_day = dt.datetime.now()-dt.timedelta(2);
path_korn = '/mnt/Z/УБПиАЭ/ОАиП/Оперативная работа/Nord Pool/Dash_daily and other'; #путь для загрузки файлов с последующим удалением

## 1. Получим данные по ожидаемым ценам на ЭЭ на NordPool

In [2]:
#Ссылка на сайт Nasdaq
link_nasdaq = "http://www.nasdaqomx.com/webproxy/DataFeedProxyIRC1.aspx";

#Что-то важное для запроса
part_change = """<param name="XPath" value="//inst[@tp='F']
[@upc &gt; 170][@upc &lt; 749][number(translate(@dft,'-',''))
&lt;= '""" + dt.datetime.now().strftime("%Y%m%d") + """' or @dft = '']"/>"""

#Запрос в Nasdaq
def post_nasdaq():
    return """<post>
    <param name="Exchange" value="NMF"/>
    <param name="SubSystem" value="Prices"/>
    <param name="Action" value="GetMarket"/>
    <param name="Market" value="GITS:NC:ENO"/>
    <param name="inst__an" value="id,tp,nm,fnm,bp,ap,lsp,spch,spchp,hp,lp,tv,rq,onexv,stlpr,oi,upc,t,exfdt,dlt,dft,tb"/>
    <param name="ext_xslt" value="nordpool-v2/inst_table_test_latest.xsl"/>
    <param name="empdata" value="false"/>
    """ + part_change + """
    <param name="ext_xslt_options" value=""/>
    <param name="ext_xslt_notlabel" value="fnm"/>
    <param name="ext_xslt_hiddenattrs" value=",not,lnot,isp,isc,exfdt,dlt,tp,dft,tb,"/>
    <param name="ext_xslt_lang" value="en"/>
    <param name="ext_xslt_tableId" value="derivatesNordicTable"/>
    <param name="ext_xslt_tableClass" value="tablesorter"/>
    <param name="ext_xslt_market" value="GITS:NC:ENO"/>
    <param name="app" value="www.nasdaqomx.com//transactions/markets/commodities/market-prices"/>
    </post>"""
    
#Загрузим данные в фрейм
nasdaq_futures = pd.DataFrame();

responce = requests.post(link_nasdaq,
                        data={'xmlquery': post_nasdaq()},
                        headers={'referer': 'http://www.nasdaqomx.com/'}
                        );
nasdaq_futures = pd.read_html(responce.text)[0];
    
#Форматируем фрейм
nasdaq_futures = nasdaq_futures.loc[nasdaq_futures.loc[:,"Product series"].str.contains("ENOAFUTBLM")];
nasdaq_futures = nasdaq_futures.loc[:, ["Product series", "Bid", "Ask", "Last", "Daily Fix"]];
nasdaq_futures.loc[:, 'Nasdaq'] = nasdaq_futures[nasdaq_futures.columns[1:]].mean(1);
nasdaq_futures = nasdaq_futures.loc[:, ("Product series", 'Nasdaq')];
nasdaq_futures.loc[:, "Product series"] = nasdaq_futures.loc[:, "Product series"].str[10:];

#Переведем дату в формат ГГГГ-ММ-ДД
format_date = lambda x: dt.datetime.strptime(x, "%b-%y");
nasdaq_futures.loc[:, "Product series"] = nasdaq_futures['Product series'].map(format_date);

nasdaq_futures = nasdaq_futures.set_index("Product series");
nasdaq_futures.index.name = 'Дата';

nasdaq_day = pd.DataFrame(index=pd.date_range(nasdaq_futures.index.min(), nasdaq_futures.index.max()+dt.timedelta(days=28), freq='D'));
nasdaq_day.index.name = 'Дата';
nasdaq_day = nasdaq_day.join(nasdaq_futures).ffill();

display(nasdaq_futures, nasdaq_day);

,Nasdaq
Дата,
2022-05-01,114.1750
2022-06-01,84.9125
2022-07-01,77.1750
2022-08-01,86.9575
2022-09-01,104.1250
2022-10-01,101.8125
2022-11-01,118.2500


,Nasdaq
Дата,
2022-05-01,114.175
2022-05-02,114.175
2022-05-03,114.175
2022-05-04,114.175
2022-05-05,114.175
...,...
2022-11-25,118.250
2022-11-26,118.250
2022-11-27,118.250


## 2. Получим данные с NordPool

In [3]:
all_link = [['price_hour_data', 'elspot-prices_', '_hourly_eur.xls'], #почасовые цены
            ['price_day_data', 'elspot-prices_', '_daily_eur.xls'], #посуточные цены
            ['price_month_data',  'elspot-prices_', '_monthly_eur.xls'], #помесячные цены
            
            ['volume_hour_data', 'elspot-volumes_', '_hourly.xls'], #почасовое потребление
            ['volume_day_data', 'elspot-volumes_', '_daily.xls'], #посуточное потребление 
            
            ['exc_hour_data', 'exchange-', '_hourly.xls'], #почасовой переток
            ['exc_day_data', 'exchange-', '_daily.xls'] #посуточный переток
           ];

#сайт НордПула
nord_link = 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/';
reg = ['fi', 'no', 'ee', 'lv', 'lt'] #страны с перетоком из/в РФ

url_of_file = {};
#Для каждого года с 2013 получим сылку на файл с сайта
for goda in range(2013, dt.datetime.now().year + 1):
    for type_dann, group_dann, form_dann  in all_link:
        #Для данных по перетоку немного иной формат записи
        if group_dann != 'exchange-':
            url_of_file[nord_link + group_dann + str(goda) + form_dann] = type_dann;
        else:
            for strana in reg:
                url_of_file[nord_link + group_dann + strana + '-connections_' + str(goda) + form_dann] = type_dann;

#Все ссылки
display(url_of_file)

{'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/elspot-prices_2013_hourly_eur.xls': 'price_hour_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/elspot-prices_2013_daily_eur.xls': 'price_day_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/elspot-prices_2013_monthly_eur.xls': 'price_month_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/elspot-volumes_2013_hourly.xls': 'volume_hour_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/elspot-volumes_2013_daily.xls': 'volume_day_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/exchange-fi-connections_2013_hourly.xls': 'exc_hour_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/exchange-no-connections_2013_hourly.xls': 'exc_hour_data',
 'https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/exchange-ee-c

In [6]:
%%time

#Обработка даты-времени
def time_form(fr, date_code):
    if 'hourly' in date_code: #Если это почасовые данные нужна доплнительная обработка
        fr['Unnamed: 0'] = fr['Hours'].str[:2] + '-' + fr['Unnamed: 0'];
        fr.drop(columns=['Hours'], inplace=True); #Удалим лишнее
        date_format = '%H-%d-%m-%Y';
    elif 'daily' in date_code:
        date_format = '%d-%m-%Y';
    elif 'monthly' in date_code:
        date_format = '%y - %b';
    
    #Создадим новый индекс - дату
    fr['Unnamed: 0'] = pd.to_datetime(fr['Unnamed: 0'], format=date_format);
    fr.set_index(['Unnamed: 0'], inplace=True);
    fr.index.name = 'Дата'
    return fr

#Загрузка данных с форматированием
def load_file(link_load):
    time_temp = str(dt.datetime.now());
    zagr_link = path_korn + f'/Load data/{time_temp} Data_temp.xls';
    zagr_link = zagr_link.replace(':', '-'); #Двоеточие недопустимый символ
    
    #Проверим существует ли ссылка
    try:
        urllib.request.urlretrieve(link_load, zagr_link);
        #Если не было ошибки, сохраним данные
        fr_html =  pd.read_html(zagr_link, skiprows=1, header=1, decimal=',', thousands='.')[0];
        fr_html = time_form(fr_html, link_load); #Обработаем дату
        os.remove(zagr_link);
        return link_load, fr_html;
    except urllib.error.HTTPError as e:
            #Если такой ссылки не существует, обработаем ошибку
            if e.code in [404]:
                print('404', link_load);
            else:
                print('Другая ошибка!');

#Запустим загрузку данных с сайта
results = process_map(load_file, url_of_file, max_workers=4);

  0%|          | 0/150 [00:00<?, ?it/s]

404 https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/exchange-ee-connections_2013_hourly.xls
404 https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/exchange-lv-connections_2013_hourly.xls
404 https://www.nordpoolgroup.com/499241/globalassets/marketdata-excel-files/exchange-lt-connections_2013_hourly.xls


ValueError: No tables found

In [5]:
#Соберем данные в несколько фреймов
data_all = {};
for i in all_link: data_all[i[0]] = pd.DataFrame();
    
for s in results:
    if not s is None: #Прове|рим, что объект существует
        if len(data_all[url_of_file[s[0]]]) == 0:
            data_all[url_of_file[s[0]]] = data_all[url_of_file[s[0]]].append(s[1]);
        else:
            #Объеденим всё. Дубликаты столбцов будут с суфикаксами
            data_all[url_of_file[s[0]]] = pd.merge(data_all[url_of_file[s[0]]], s[1], how='outer', on='Дата', suffixes=('', '_delme'));
                
for fr_type in data_all.keys():
    #Удалим дубликаты столбцов
    suf_col = [x for x in data_all[fr_type].columns if '_delme' not in x];
    #Добавим значения из колонок с суффиксами в колонки без суффиксов
    for clear_col in suf_col:
        all_col = [y for y in data_all[fr_type].columns if clear_col in y]
        data_all[fr_type][clear_col] = data_all[fr_type][all_col].mean(1);
        
    #И удалим их
    data_all[fr_type] = data_all[fr_type][suf_col];    

NameError: name 'results' is not defined

# Страны

In [ ]:
naming_dict = {'FI': 'Финляндия', 'LT': 'Литва', 'LV': 'Латвия', 'EE': 'Эстония',
               'NO': 'Норвегия', 'SE': 'Швеция', 'DK': 'Дания',
               'DE-LU': 'Германия', 'BE': 'Бельгия', 'AT': 'Австрия',
               'FR': 'Франция',  'NL': 'Нидерланды', 'PL': 'Польша'};

# Обработка цен

In [ ]:
#Блок обработки входных данных
col_obr = {'NO': ['Bergen', 'Kr.sand', 'Molde', 'Oslo', 'Tr.heim', 'TromsÃ¸'],
           'DK': ['DK1', 'DK2'],
           'SE': ['SE1', 'SE2', 'SE3', 'SE4']};

In [ ]:
def form_price(fr):
    """
    Функция для обработки данных по ценам.
    На входе не обработанный фрейм, формат даты, признак почасовых данных.
    На выходе фрейм
    """
    fr.drop(columns=['SYS', 'ELE', 'FRE'], inplace=True, errors='ignore'); #Удалим лишнее
    
    #Обработаем названия-дубликаты
    for reg in [i for i in fr.columns if '.' != i[-2]]:
        col_reg = [i for i in fr.columns if reg in i];
        fr[reg] = fr[col_reg].mean(1);
        col_reg.remove(reg)
        fr.drop(columns=col_reg, inplace=True);
        
    #Обработаем страны, разделенные на несколько ценовых зон
    for part_reg in ['SE', 'DK']:
        col_part = [part_reg+str(i) for i in range(1,10)];
        col_part = [i for i in col_part if i in fr.columns];
        fr[part_reg] = fr[col_part].mean(1);
        fr.drop(columns=col_part, inplace=True);
    
    #Обработаем регионы Норвегии
    no_reg = ['Bergen', 'Kr.sand', 'Molde', 'Oslo', 'Tr.heim', 'TromsÃ¸'];
    fr['NO'] = fr[no_reg].mean(1);
    fr.drop(columns=no_reg, inplace=True);
    
    #Переведем всё на русский
    fr.rename(columns=naming_dict, inplace=True);
    fr = fr[[i for i in naming_dict.values() if i in fr.columns]];
    return fr

#### Почасовые цены

In [ ]:
data_all['price_hour_data'] = form_price(data_all['price_hour_data']);
#Группировка нужна, чтобы избавиться от дубликатов при переходе с зимнего времени на летнее и наоборот
data_all['price_hour_data'] = data_all['price_hour_data'].groupby(data_all['price_hour_data'].index).agg('first');
#По причине смена зимнего-летнего времени заполним данные
#Сделаем это только для предыдущего периода
data_all['price_hour_data'].loc[:l_day] = data_all['price_hour_data'].loc[:l_day].ffill();
data_all['price_hour_data']

#### Посуточные цены

In [ ]:
data_all['price_day_data'] = form_price(data_all['price_day_data']);
data_all['price_day_data']

#### Помесячные цены

In [ ]:
data_all['price_month_data'] = form_price(data_all['price_month_data']); #дата в помесячных данных ГГ - ММ
data_all['price_month_data']

# Обработка объемов купли-продажи

In [ ]:
def form_volume(fr):
    """
    Функция для обработки данных по объемам покупки.
    На входе не обработанный фрейм, формат даты, признак почасовых данных.
    На выходе фрейм
    """    
    fr.drop(columns=['Turnover at system price', 'ELE Buy', 'ELE Sell',
                     'FRE Buy', 'FRE Sell', 'SE', 'DK'], inplace=True, errors='ignore'); #Удалим лишнее
    
    for code in ['FI', 'EE', 'LT', 'LV', 'NO', 'SE', 'DK']:
        code_col = [i for i in fr.columns if 'buy' in i.lower() and code==i[:2]]; #Обработаем только цены покупки
        other_col = [i for i in fr.columns if ('buy' in i.lower() or 'sell' in i.lower()) and code==i[:2]]; #Продажи
        fr[code] = fr[code_col].sum(axis=1);
        fr.drop(columns=other_col, inplace=True);
    
    #Переведем всё на русский
    fr.rename(columns=naming_dict, inplace=True);
    fr = fr[[i for i in naming_dict.values() if i in fr.columns]];
    return fr

#### Почасовые объемы купли-продажи, млн.кВтч

In [ ]:
data_all['volume_hour_data'] = form_volume(data_all['volume_hour_data']).applymap(lambda x: np.NaN if x==0 else x)/1000;
#Группировка нужна, чтобы избавиться от дубликатов при переходе с зимнего времени на летнее и наоборот
data_all['volume_hour_data'] = data_all['volume_hour_data'].groupby(data_all['volume_hour_data'].index).agg('first');
#По причине смена зимнего-летнего времени заполним данные
#Сделаем это только для предыдущего периода
data_all['volume_hour_data'].loc[:l_day] = data_all['volume_hour_data'].loc[:l_day].ffill();
data_all['volume_hour_data']

#### Посуточные объемы купли-продажи, млн.кВтч

In [ ]:
data_all['volume_day_data'] = form_volume(data_all['volume_day_data']).applymap(lambda x: np.NaN if x==0 else x)/1000;
data_all['volume_day_data']

#### Помесячные объемы купли-продажи, млн.кВтч

In [ ]:
#Видно, что почасовые данные посуточными
data_all['volume_hour_data'].groupby(data_all['volume_hour_data'].index.astype(str).str[:10]).sum()

In [ ]:
data_all['volume_month_data'] = data_all['volume_hour_data'].groupby(data_all['volume_hour_data'].index.astype(str).str[:7]).sum();
data_all['volume_month_data'].index = pd.to_datetime(data_all['volume_month_data'].index, format='%Y-%m');
data_all['volume_month_data']

# Обработка перетока

In [ ]:
def form_exchange(fr):
    """
    Функция для обработки данных по перетокам.
    На входе не обработанный фрейм, формат даты, признак почасовых данных.
    На выходе фрейм
    """    
    exc_col = [i for i in fr.columns if 'RU' in i];
    fr = fr[exc_col];
    col_new_name = {i: i[:2] for i in fr.columns};
    fr = fr.rename(columns=col_new_name);
    
    #Переведем всё на русский
    fr.rename(columns=naming_dict, inplace=True);
    fr = fr[[i for i in naming_dict.values() if i in fr.columns]];
    return fr

#### Почасовый перток, МВт

In [ ]:
data_all['exc_hour_data'] = form_exchange(data_all['exc_hour_data']);
#Группировка нужна, чтобы избавиться от дубликатов при переходе с зимнего времени на летнее и наоборот
data_all['exc_hour_data'] = data_all['exc_hour_data'].groupby(data_all['exc_hour_data'].index).agg('first');
#По причине смена зимнего-летнего времени заполним данные
#Сделаем это только для предыдущего периода
data_all['exc_hour_data'].loc[:l_day] = data_all['exc_hour_data'].loc[:l_day].ffill();

data_all['exc_hour_data']['NordPool'] = data_all['exc_hour_data'].sum(1);
data_all['exc_hour_data']

#### Посуточный перток, МВт

In [ ]:
data_all['exc_day_data'] = form_exchange(data_all['exc_day_data'])/24;

data_all['exc_day_data']['NordPool'] = data_all['exc_day_data'].sum(1);
data_all['exc_day_data']

In [ ]:
#Видно, что почасовые данные по перетоку совпадают с посуточными
#Будем использовать почасовые, так как данных больше
data_all['exc_hour_data'].groupby(data_all['exc_hour_data'].index.astype(str).str[:10]).mean()

In [ ]:
data_all['exc_day_data'] = data_all['exc_hour_data'].groupby(data_all['exc_hour_data'].index.astype(str).str[:10]).mean();
data_all['exc_day_data'].index = pd.to_datetime(data_all['exc_day_data'].index, format='%Y-%m-%d');
data_all['exc_day_data']

#### Помесячный перток, МВт

In [ ]:
data_all['exc_month_data'] = data_all['exc_hour_data'].groupby(data_all['exc_hour_data'].index.astype(str).str[:7]).mean();
data_all['exc_month_data'].index = pd.to_datetime(data_all['exc_month_data'].index, format='%Y-%m');
data_all['exc_month_data']

# Получение помесячной цены за последний месяц

In [ ]:
month_price_calc = pd.concat([data_all['price_day_data'],
           data_all['volume_day_data']], keys=['Цена', 'Объем'], axis=1)
month_price_calc

In [ ]:
col_exsist = month_price_calc['Объем'].columns
small_price = month_price_calc.loc[:, (slice(None), col_exsist)].copy(deep=True)

for i in col_exsist:
    small_price[('Стоимость', i)] = small_price[('Цена', i)] * small_price[('Объем', i)];
    
small_price_gr = small_price.groupby(small_price.index.astype(str).str[:7]).sum();

for i in col_exsist:
    small_price_gr[('Цена', i)] = small_price_gr[('Стоимость', i)] / small_price_gr[('Объем', i)];

small_price_gr.index = pd.to_datetime(small_price_gr.index, format='%Y-%m');
small_price_gr

In [ ]:
#Заполним данные по цене за текущий месяц
idx_nan = data_all['price_month_data'].index.max();
#Если данных нет, то выполним заполнение
if data_all['price_month_data'].loc[idx_nan].isna().any():
    for country in small_price_gr['Цена'].columns:
        data_all['price_month_data'].loc[idx_nan, country] = small_price_gr.loc[idx_nan, ('Цена', country)];

data_all['price_month_data']

# Сбор данных вместе

### Почасовые данные

In [ ]:
hour_data = pd.concat([data_all['price_hour_data'],
                       data_all['volume_hour_data'],
                       data_all['exc_hour_data']
                      ], keys=['Цена', 'Объем', 'Переток'], axis=1);

hour_data

### Посуточные данные

In [ ]:
day_data = pd.concat([data_all['price_day_data'],
                      data_all['volume_day_data'],
                      data_all['exc_day_data'],
                      nasdaq_day.loc[data_all['price_day_data'].index.max():]
                     ], keys=['Цена', 'Объем', 'Переток', 'Nasdaq'], axis=1);

day_data.loc[data_all['price_day_data'].index.max(), ('Nasdaq', 'Nasdaq')] = day_data.loc[data_all['price_day_data'].index.max(), ('Цена', 'Финляндия')];
day_data

### Помесячные данные

In [ ]:
month_data = pd.concat([data_all['price_month_data'],
                       data_all['volume_month_data'],
                       data_all['exc_month_data'],
                       nasdaq_futures
                      ], keys=['Цена', 'Объем', 'Переток', 'Nasdaq'], axis=1);

month_data

# Сохраним результаты

In [ ]:
#В эксель
with pd.ExcelWriter(path_korn + '/Clear data/All_data.xlsx') as writer:
    hour_data.to_excel(writer, sheet_name='Час');
    day_data.to_excel(writer, sheet_name='День');
    month_data.to_excel(writer, sheet_name='Месяц');

#В пиклу
saveObject = (hour_data, day_data, month_data)
with open(path_korn + '/Clear data/All_data.pickle', 'wb') as f:
    pickle.dump(saveObject, f);